### Тестовое задание № 2. 

Предположим, что речь идет о модели ребаланса, реализованный в Whoosh (подробнее [здесь](https://habr.com/ru/companies/whoosh/articles/720194/)). Основной фокус этой модели - прогнозирование количества стартов / финишев из любой парковки, где распологаются самокаты;  алгоритм модели - Prophet. 

Задачи, которые ставятся перед разработчиком для релиза модели в продаакш: 
1. Реализовать обучение модели на качественных данных, продумать логику препроцессинга данных для потокового инфрененса модели; 
2. Перенести модель в продакшн сервис;  
2. Реализовать обучение / переобучение / дообучение на поток; 
3. Реализовать мониторинг моделей;   
4. Реализовать тестирование моделей. 

Предположим код лежит в ветки master, помечанный тегом для релиза, а так же на этот тег навешаны версии бибилиотек сервиса, в котором будет развернута модель. Модель представляет собой `.py` файлы, разбитые на data_loading, data_preprocessing, model, model_testing, а так же баш скрипты для ранинга отдельных кусков модели (к примеру, тот же препроцессинг данных) или к примеру покрытия тестами. Так же было бы неплохо сохранить веса лучшей модели в репосе (например, в виде pkl файлов). Финальное - Dockerfile, который позволит развернуть код в контейнере. 

Далее я бы сформировала 2 пайплайна в виде направленного циклического графа (DAG); первый из них отвечал бы за feature engineering, тк в процессе дополнительного анализа / доработок модели могут быть вытащены дополнительные признаки модели, второй - automated ml workflow, собственно процесс для запуска кода модели для инференса / переобучения / дообучения модели.

1. Feature engineering pipeline.   
Этот DAG я бы сформировала из следующих tasks: connection to raw data (API подключения к хранилищу, к примеру hdfs), data extraction (загрузка данных), data transformation (препроцессинг данных на основе EDA), feature engineering (выявление признаков, которые мы будем использовать в модели). Data sourses-ами (row data) для задачи предсказания временных рядов выступает straming data (например, исторические агрегированные данные по поездкам (по 30 минут)), batch data (например, данные по парковкам - актуальность парковок, их популярность), cloud data.  На выходе из Daga подготовленные данные отправляются Feature Storage (БД). Данные для модели представляют собой временные ряды, поэтому стрименговые данные записываются в БД инкрементально (append only), если мы говорим про batch данные, то в зависимости от БТ модели (например, используется информация по уже не существующим парковкам для трейна модели) они перезаписываются / добавляются инкрементально. 

2. Automated ml workflow pipeline.   
Пусть такой DAG состоит из стандартных тасков:  data extraction (из feature store), data preparation / validation (под модель), training / running model; model validation, model exporting, pushing model to registry; каждый task имеет параметры, которые идентифицируют бизнес логику, и логи, которые отображаются в output, а сам же output и прочие артефакты (как модели, так и данных) могут записываться в локальное распределенное храниоище s3, а выход модели в hdfs.

Информация по состоянию парковок приходит каждые 30 минут, встает вопрос, как часто запускать модель на перерасчет. Это можно делать те же каждые 30 минут, но всё зависит от ресурсоёмкости сервиса. Если есть ограничения по памяти / нагрузке, то я бы смотрела на то, как хорошо, модель предсказывает данные на периоде 30, 60, 90, 120 ... минут от начальной точки, и если метрики устраивают меня и заказчика, то определилась бы с более широким диапазоном по времени запуска модели.

Самый популярный опен-сорс-сервис для mlops задач это mlfow с 4-мя компонентами: mlflow tracking для логирования и отслеживания графиков, метрик, ак же у него есть API для вывода информации в Grafana, к примеру; mlflow models для упаковки моделей в контейнеры, как раз таки он отчвечает за периодический запуск модели, тк по REST API данные-результат модели можно выкатить в сервис для курьеров самокатов; mlflow regisry для контроля жизненным циклом модели, если модель нужно реализоваать на стороннем сервере, regisry в помощь; и последнее mlflow projects для упаковски кода и его переиспользования.



